# Machine Vision 2024/25 - Assignment 5: Camera Calibration

In this assignment we will calibrate a camera using Tsai's camera calibration.

In [ ]:
import copy
import math
import matplotlib.pyplot as plt
import numpy as np
from skimage.io import imread

#### Tsai's Camera Calibration

The file `res/tsai/calibration.png` contains an image that shows a calibration scene with 16 calibration markers.
The position of the markers in world coordinates is stored in the array `var: (world_coordinates)`, while the positions of the image coordinates are stored in the array `var: (image_coordinates)`.

In [ ]:
image = imread("res/tsai/calibration.png")

# Dimensions: [X, Y, Z] (xi, eta, zeta)
world_coordinates = np.array([
    [0.155, 0.301, 0],
    [0.720, 0.171, 0],
    [1.283, 0.171, 0],
    [1.934, 0.171, 0],
    [0.156, 1.194, 0],
    [0.156, 1.879, 0],
    [0.156, 2.552, 0],
    [1.110, 2.028, 0],
    [1.833, 1.258, 0],
    [0.192, 2.952, 0.602],
    [1.177, 2.952, 0.604],
    [1.979, 2.703, 0.615],
    [1.979, 1.817, 0.615],
    [1.979, 1.419, 0.615],
    [1.979, 0.433, 0.617],
    [0.899, 0.920, 0],
])

# Dimensions: [v, u]
image_coordinates = np.array([
    [418, 135],
    [398, 307],
    [355, 436],
    [314, 552],
    [282, 81],
    [215, 56],
    [168, 39],
    [178, 216],
    [214, 393],
    [54, 29],
    [54, 178],
    [42, 300],
    [74, 370],
    [93, 411],
    [151, 530],
    [280, 262],
])

##### Preparation

Visualize the image and the image coordinate of each marker.

In [ ]:
def plot_coordinates(*, image, image_coordinates, color):
    # @student: visualize the images and highlight the image coordinates in red
    #           the function should also return the marked image, as we will use it again later on
    image_marked = copy.deepcopy(image)

    ...

    return image_marked

image_marked = plot_coordinates(image=image, image_coordinates=image_coordinates, color=[255,0,0])

##### Calculation

In [ ]:
# This function calculates the eigenvector belonging to the smallest eigenvalue
def calculate_smallest_eigenvec(*, M):
    evals, evecs = np.linalg.eig(M)
    eigen_sorted = np.argsort(evals)
    eval = evals[eigen_sorted[0]]
    evec = evecs[:, eigen_sorted[0]]

    return evec, eval

In [ ]:
def tsai(*, world_coordinates, image_coordinates):
    assert world_coordinates.shape[0] == image_coordinates.shape[0], "Each world coordinate must have a related image coordinate"
    num_matches = world_coordinates.shape[0]

    W = np.ones(shape=[num_matches, 4])
    W[:,:3] = world_coordinates

    # @student: build overdetermined coordinate system C based on (S, Su, Sv, Suv)
    S = ...
    Su = ...
    Sv = ...
    Suv = ...

    # For each match calculate S, Su, Sv and Suv
    for i in range(num_matches):
        s = ...
        S += ...
        Su += ...
        Sv += ...
        Suv += ...

    C = np.zeros(shape=[12,12])
    C[0:4, 0:4] = S
    C[4:8, 4:8] = S
    C[0:4, 8:12] = -Su
    C[8:12, 0:4] = -Su
    C[4:8, 8:12] = -Sv
    C[8:12, 4:8] = -Sv
    C[8:12, 8:12] = Suv

    # find the eigenvector corresponding to the smallest eigenvalue
    evec, eval = calculate_smallest_eigenvec(M=C)


    # Scale the eigenvector based on the fact that M[3,0:3] is equal to one row of the rotation matrix
    # Therefore, its euclidean length must be 1
    # However, there are still two possibilities either -evec / ... or + evec/...
    # We can choose the correct one by checking the determinant of the rotation matrix
    # @student: Scale +/-eigenvector
    evec_pos = ...
    evec_neg = ...

    found = False
    # evaluate +/- eigenvector
    for evec in [evec_pos, evec_neg]:
        # @student: reshape eigenvector to M[3, 4]
        M = ...

        #  @student: define output matrices, A, R, t
        A = ...
        R = ...
        t = ...

        # @student: determine A,R, t parameter from M
        R[2,:] = ...
        t[0, 2] = ...
        v0 = ...
        u0 = ...

        beta_pp = ...
        t[0,1] = ...
        R[1,:] = ...
        gamma_pp = ...

        alpha_pp = ...
        R[0, :] = ...
        t[0,0] = ...

        A[0,0] = ...
        A[0,1] = ...
        A[1,1] = ...
        A[0,2] = ...
        A[1,2] = ...
        A[2,2] = ...

        if math.isclose(1, np.linalg.det(R), abs_tol=0.001):
            found = True
            break

    assert found == True, "Either the positive nor negative rotation matrix must have a determinant of 1"
    return A, R, t

A, R, t = tsai(world_coordinates=world_coordinates, image_coordinates=image_coordinates)

##### Evaluation

In order to evaluate the calibration, we can project our known world points into the image and calculate the reprojection error.

First we need to transform our coordinated and parameters to homogeneous form.

In [ ]:
# @student: turn A, R, t into homogeneous form (var: A_h, Rt_h) and transform world coordinates into homogeneous form (var: WC_h)
Rt_h = np.zeros(shape=[4,4])
Rt_h[...] = ...

A_h = np.zeros(shape=[4,4])
A_h[...] = ...

WC_h = np.zeros(shape=[16, 4])
WC_h[...] = ...

then we can project the world coordinated to image space.


$
z * \begin{bmatrix}u' \\ v' \\ 1\end{bmatrix} = A_h * Rt_h * \begin{bmatrix} \xi \\ \eta \\ \zeta \\ 1\end{bmatrix},
$ where $A_h$ and $Rt_h$ are $A$ and $Rt$ in homogeneous form.

In [ ]:
# @student: Calculate 2D projection using A_h, Rt_h and WC_h
projection = ...

# hint: the projection returns u -> 0 and v -> 1
# possible way to fix this:
# projection = np.fliplr(projection.transpose())

In [ ]:
image_marked = plot_coordinates(image=image_marked, image_coordinates=projection.astype(np.int32), color=[0,255,0])

image_marked = plot_coordinates(image=image_marked, image_coordinates=projection.astype(np.int32), color=[0,255,0])
Once the projected pixel $p'$ ($u'$, $v'$) is calculated based on the calibration parameters it can be compared with the known pixel $p$ ($u$, $v$).

The resulting error is called reprojection error $e_r$, which can be calculated using the equation:

$e_r = \frac{1}{N} * \sum{|p - p'|_2}$, where N is the number of pixels and ||_2 is the L2-Norm.

In [ ]:
def calculate_reprojection_error(*, p, p_pred) -> float:
    # @student: calculate the reprojection error
    return ...

calculate_reprojection_error(p=image_coordinates, p_pred=projection)